In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print(ws)

In [16]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


Creating a new compute cluster...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-12-20T12:26:41.190000+00:00', 'errors': None, 'creationTime': '2020-12-20T12:26:38.177568+00:00', 'modifiedTime': '2020-12-20T12:26:53.948054+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
# from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)



In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")


RunId: HD_fda34223-a94c-456b-8bf7-52e84aa1d17e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_fda34223-a94c-456b-8bf7-52e84aa1d17e?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-131112/workspaces/quick-starts-ws-131112

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-20T12:06:58.894473][API][INFO]Experiment created<END>\n""<START>[2020-12-20T12:06:59.402645][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-12-20T12:07:00.0064856Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-20T12:06:59.575960][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_fda34223-a94c-456b-8bf7-52e84aa1d17e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_fda34223-a94c-456b-8bf7-52e84aa1d17e?wsid=/subscriptions/a24a2

In [5]:
import joblib

# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

# Get your best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
# get_file_names()
# Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())
# log_accuracy_table()
# Logs an accuracy table to the artifact store. Maybe use it for testing.
# log_accuracy_table(name, value, description='')



[{'run_id': 'HD_fda34223-a94c-456b-8bf7-52e84aa1d17e_14', 'hyperparameters': None, 'best_primary_metric': 0.9176024279210926, 'status': 'Completed'}, {'run_id': 'HD_fda34223-a94c-456b-8bf7-52e84aa1d17e_0', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_fda34223-a94c-456b-8bf7-52e84aa1d17e_2', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_fda34223-a94c-456b-8bf7-52e84aa1d17e_15', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_fda34223-a94c-456b-8bf7-52e84aa1d17e_7', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_fda34223-a94c-456b-8bf7-52e84aa1d17e_6', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_fda34223-a94c-456b-8bf7-52e84aa1d17e_4', 'hyperparameters': None, 'best_primary_metric': 0.916

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [13]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py

automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)


In [17]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_31743594-d9a8-4152-9c36-2f432987b6ca,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_31743594-d9a8-4152-9c36-2f432987b6ca',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-11-06T13:56:48.122467Z',
 'endTimeUtc': '2023-11-06T14:22:31.468596Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 15 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-243731","workspace_name":"quick-starts-ws-243731","region":"westeurope","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_s

In [18]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())


Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: udacity-project,
Id: AutoML_31743594-d9a8-4152-9c36-2f432987b6ca_22,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('18', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.1, gamma=0.1, max_depth=9, max_leaves=511, n_estimators=25, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type

In [19]:
best_run.get_metrics()

{'recall_score_weighted': 0.9164188163884674,
 'f1_score_weighted': 0.9097224638164556,
 'recall_score_macro': 0.7239314682454665,
 'matthews_correlation': 0.5263762603240173,
 'recall_score_micro': 0.9164188163884674,
 'average_precision_score_macro': 0.8246580847992477,
 'average_precision_score_micro': 0.9814321747714827,
 'precision_score_micro': 0.9164188163884674,
 'balanced_accuracy': 0.7239314682454665,
 'log_loss': 0.18894093566567532,
 'AUC_weighted': 0.9478787189939635,
 'precision_score_weighted': 0.9079156261992071,
 'norm_macro_recall': 0.447862936490933,
 'AUC_micro': 0.9806426346075467,
 'AUC_macro': 0.9478787189939635,
 'average_precision_score_weighted': 0.955437507348456,
 'f1_score_macro': 0.7570452807215802,
 'f1_score_micro': 0.9164188163884674,
 'accuracy': 0.9164188163884674,
 'precision_score_macro': 0.8096233623158824,
 'weighted_accuracy': 0.9642319376728848,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_31743594-d9a8-4152-9c36-2f432987b6ca_2

In [20]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mn...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('18', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.1, gamma=0.1, max_depth=9, max_leaves=511, n_estimators=25, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg

In [21]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_31743594-d9a8-4152-9c36-2f432987b6ca_22,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
# Cluster clean up

compute_target.delete()